In [1]:
import geemap
import ee

In [2]:
ee.Authenticate()


Successfully saved authorization token.


Initialize Earth Engine

In [3]:
ee.Initialize()

Load dataset imagery and define pre and post wildfire period

In [4]:
# Load the feature collection
geometry = ee.FeatureCollection("projects/ee-lorenzocarlassara/assets/rabbitFire")

# Define variables
aoi_name = 'Riverside'
pre_date_start = '2023-06-13'
pre_date_end = '2023-07-13'
post_date_start = '2023-07-23'
post_date_end = '2023-08-13'
point = ee.Geometry.Point(-117.070916, 33.889467)
start = '2023-07-14'
end = '2023-07-22'
ndwiThreshold = 0.45

# Center the map on the point
Map = geemap.Map()
Map.centerObject(point, 11)

Define the Area of Interest

In [5]:
# Load the admin2 feature collection
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

# Filter the admin2 feature collection based on the ADM2 name
aoi = admin2.filter(ee.Filter.eq('ADM2_NAME', aoi_name))

# Create visualization parameters
vis_params = {'color': 'red'}

# Add the AOI layer with the specified visualization parameters
Map.addLayer(aoi, vis_params, aoi_name, 0)

# Display the map
Map

Map(center=[33.88946700000001, -117.070916], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

FIRMS & MODIS

In [6]:
# Load FIRMS dataset
dataset = (ee.ImageCollection("FIRMS")
           .select(['T21'])
           .filterDate(start, end))

# Visualization parameters for FIRMS data
firesVis = {
    'min': 325.0,
    'max': 400.0,
    'palette': ['yellow', 'orange', 'red']
}

Map.addLayer(dataset, firesVis, 'Fires')

Landsat-8

In [7]:
# Load LANDSAT-8 Collection and apply filters
LS8Collection = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
                 .filter(ee.Filter.lt('CLOUD_COVER_LAND', 10))
                 .filterBounds(aoi))

# Calculate pre-fire and post-fire median images
LS8_prefire = LS8Collection.filterDate(pre_date_start, pre_date_end).median()
LS8_postfire = LS8Collection.filterDate(post_date_start, post_date_end).median()

# Visualization parameters for LANDSAT-8 RGB
LS8_rgbVis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B5', 'B4', 'B3']
}

Define the function to mask clouds from Sentinel-2 images

In [8]:
# Define the maskS2clouds function
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

Define the pre and post wildfire images from Sentinel-2

In [9]:
# Define Sentinel-2 collection and filter conditions
SE2Collection = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                .filterBounds(aoi)

# Define prefire and postfire images
prefire = SE2Collection \
          .filterDate('2023-06-13', '2023-07-13') \
          .map(maskS2clouds) \
          .median()

postfire = SE2Collection \
           .filterDate('2023-07-23', '2023-08-13') \
           .map(maskS2clouds) \
           .median()

# Define the SimpleStyle dictionary for visualization
rgb_vis = {'bands': ['TCI_R', 'TCI_G', 'TCI_B']}

# Define visualization parameters
fireRGB_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B8', 'B4', 'B3']}

# Add layers to the map
Map.addLayer(postfire, rgb_vis, 'PostFireRGB')
Map.addLayer(prefire, rgb_vis, 'PreFireRGB')
Map.addLayer(postfire, fireRGB_vis, 'PostFireFRGB')
Map.addLayer(prefire, fireRGB_vis, 'PreFireFRGB')

Define Normalized Differenfe Water Index to mask water from MSI images

In [10]:
# Define the NDWI calculation function
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI')
    return ndwi.lt(ndwi_threshold)

# Define the NDWI threshold
ndwi_threshold = 0.0

# Calculate NDWI for pre-fire and post-fire images
water_prefire = calculate_ndwi(LS8_prefire)
water_postfire = calculate_ndwi(LS8_postfire)

# Update water masks and images
water_mask_prefire = prefire.updateMask(water_prefire)
water_mask_postfire = postfire.updateMask(water_postfire)
LS8_prefire = LS8_prefire.updateMask(water_prefire)
LS8_postfire = LS8_postfire.updateMask(water_postfire)
prefire = prefire.updateMask(water_prefire)
postfire = postfire.updateMask(water_postfire)

Compute Normalized Burn Ratio for pre-fire and post-fire images of Sentinel-2 and Landsat-8

In [11]:
# Calculate NBR for pre-fire and post-fire images
postNBR = postfire.normalizedDifference(['B8', 'B12']).select([0], ['NBR'])
preNBR = prefire.normalizedDifference(['B8', 'B12']).select([0], ['NBR'])
d_NBR = preNBR.subtract(postNBR)
dNBR = d_NBR.multiply(1000)

# Calculate NBR for LANDSAT-8 pre-fire and post-fire images
LS8_postNBR = LS8_postfire.normalizedDifference(['B5', 'B7']).select([0], ['LS8 NBR'])
LS8_preNBR = LS8_prefire.normalizedDifference(['B5', 'B7']).select([0], ['LS8 NBR'])
LS8_d_NBR = LS8_preNBR.subtract(LS8_postNBR)
LS8_dNBR = LS8_d_NBR.multiply(1000)

# Define color palette for NBR
nbr_vis = {'min': -0.2, 'max': 0.6, 'palette': ['red', 'orange', 'yellow', 'green', 'darkgreen']}

# Add layers to the map
Map.addLayer(postNBR, nbr_vis, 'PostFireNBR')
Map.addLayer(preNBR, nbr_vis, 'PreFireNBR')

Compute Burned Area Index for Sentinel-2 images

In [12]:
# Burned Area Index for Sentinel-2 function
def compute_bais2(image):
    B4 = image.select('B4')  # Red
    B6 = image.select('B6')  # Red Edge 2
    B7 = image.select('B7')  # Red Edge 3
    B8A = image.select('B8A')  # Red Edge 4
    B12 = image.select('B12')  # Short Wave Infrared 2

    term1 = (B6.multiply(B7).multiply(B8A)).sqrt().divide(B4.sqrt())
    term2 = B12.subtract(B8A).divide((B12.add(B8A)).sqrt()).add(1)

    BAIS2 = ee.Image(0).subtract((ee.Image(1).subtract(term1)).multiply(term2))

    return BAIS2.rename('BAIS2')

In [14]:
# Calculate BAIS2 for pre-fire and post-fire images
postBAIS2 = compute_bais2(postfire)
preBAIS2 = compute_bais2(prefire)
d_BAIS2 = preBAIS2.subtract(postBAIS2)
dBAIS2 = d_BAIS2.multiply(1000)

# Define visualization parameters for dNBR and dBAIS2 layers
grey_palette = ['white', 'black']
# Define visualization parameters for dNBR and dBAIS2 layers
grey_vis = {'min': -1000, 'max': 1000, 'palette': grey_palette}

# Add layers to the map
Map.addLayer(dNBR, grey_vis, 'dNBR greyscale')
Map.addLayer(dBAIS2, grey_vis, 'dBAIS2 greyscale')

# Define SLD intervals for styling
sld_intervals = '''
<RasterSymbolizer>
  <ColorMap type="intervals" extended="false">
    <ColorMapEntry color="#ffffff" quantity="-1000" label="-500"/>
    <ColorMapEntry color="#7a8737" quantity="-500" label="-250"/>
    <ColorMapEntry color="#acbe4d" quantity="-300" label="-100"/>
    <ColorMapEntry color="#0ae042" quantity="135" label="100"/>
    <ColorMapEntry color="#fff70b" quantity="270" label="270"/>
    <ColorMapEntry color="#ffaf38" quantity="440" label="440"/>
    <ColorMapEntry color="#ff641b" quantity="660" label="660"/>
    <ColorMapEntry color="#a41fd6" quantity="2000" label="2000"/>
  </ColorMap>
</RasterSymbolizer>
'''

# Apply SLD intervals to the layers and add to the map
Map.addLayer(dBAIS2.sldStyle(sld_intervals), {}, 'SE2 dBAIS2 classified')
Map.addLayer(LS8_dNBR.sldStyle(sld_intervals), {}, 'LS8 dNBR classified')
Map.addLayer(dNBR.sldStyle(sld_intervals), {}, 'SE2 dNBR classified')

In [20]:
# Clip dNBR to the wildfire aoi geometry
clipped_dNBR = dNBR.clip(geometry)

# Extract only burned pixels
burned_area = clipped_dNBR.gte(100)

# Get pixel area of the affected burned layer
burned_pixel_area = burned_area.multiply(ee.Image.pixelArea())  # Calculate the area of each pixel

# Sum pixels of affected burned layer
burned_stats = burned_pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),  # Sum all pixels with area information
    geometry=aoi,
    scale=30,
    maxPixels=1e9
)

# Print burned_stats
print(burned_stats.getNumber('NBR').getInfo(), 'm^2')

# Calculate burned area in hectares and acres
burned_area_ha = burned_stats.getNumber('NBR').divide(10000).round()
burned_area_ac = burned_stats.getNumber('NBR').divide(4046.86).round()

# Print burned area in hectares and acres
print('Burned area:', burned_area_ha.getInfo(),'hectares', '(',burned_area_ac.getInfo(),'acres)')

31869646.97198032 m^2
Burned area: 3187 hectares ( 7877 acres)


In [21]:
# Define the legend panel for dNBR classes
legend = geemap.legend(outlines=['#7a8737', '#acbe4d', '#0ae042', '#fff70b', '#ffaf38', '#ff641b', '#a41fd6', '#ffffff'],
                       labels=['Enhanced Regrowth, High', 'Enhanced Regrowth, Low', 'Unburned', 'Low Severity',
                               'Moderate-low Severity', 'Moderate-high Severity', 'High Severity', 'NA'],
                       title='dNBR Classes')

# Define the results panel
results = geemap.panel([
    'Results',
    f'{aoi_name} wildfire period:',
    f'{start} and {end}',
    'Estimated burned area:',
    'based on Sentinel-2 MSI (20m)',
    'Please wait...',
    'Script produced by: Lorenzo Carlassara',
    'Politecnico di Milano 2023-08-24'
], widget_type='label')

# Create a map and add legend and results panels
Map = geemap.Map()
Map.add_legend(legend, position='bottomright')
Map.add_panel(results, position='bottomleft', width='350px')
Map.centerObject(geometry, 11)

AttributeError: module 'geemap' has no attribute 'legend'